# WindFarmer automation demo
1. Set up a workbook
2. Compete AEP in an existing workbook
3. Export the web API inputs and call the API
4. Call the web API many times

# In app automation
## Scripted project setup
* Open a new workbook and save it somewhere
* Import the script "Scripted project setup": 
  ```C:\Repos\WindFarmerAutomation\Examples\InApp\Scripted project setup\Scripted Project Setup.cs```
* Edit the path to the Hawaii demodata folder ```dataPath```
* Run the script to construct a Hawaii demo workbook

## SDK automation

### First set up the connection to the SDK
This effectively opens the desktop windfarmer application in invisible mode

In [1]:
#%% Setup connection to the WFA SDK
print('Connecting to the WindFarmer SDK')
import sdk
windfarmer_installation_folder = r'C:\Program Files\DNV\WindFarmer - Analyst 1.6.4'
wf = sdk.Sdk(windfarmer_installation_folder)
print(' > SDK is now up and running!')

Connecting to the WindFarmer SDK
 > SDK is now up and running!


## Compute AEP in windfarmer desktop, via the SDK

Some folders for inputs and results - using relative paths within this repository

In [2]:
import os
# script_dir_name = os.path.dirname(__file__) # Use this if running as python script, not notebook
script_dir_name = os.path.curdir
root_dir = os.path.abspath(os.path.join(script_dir_name, '..', '..'))
workbook_folder_path = os.path.join(root_dir, 'DemoData', 'OffshoreBalticCoast')
workbook_file_path = os.path.join(workbook_folder_path, 'BalticCoast.wwx')

Open the WindFarmer wwx workbook


In [3]:
print( "opening windfarmer workbook..." +  workbook_file_path)
wf.Toolbox.OpenWorkbook(os.path.join(workbook_folder_path, workbook_file_path))
print("> WindFarmer file opened: " + wf.Toolbox.get_CurrentWorkbookPath() + "\n")

opening windfarmer workbook...c:\Repos\WindFarmer-automation-github\DemoData\OffshoreBalticCoast\BalticCoast.wwx
> WindFarmer file opened: c:\Repos\WindFarmer-automation-github\DemoData\OffshoreBalticCoast\BalticCoast.wwx



Read some project data:


In [4]:
turbine_count = wf.Workbook.Turbines.Count
non_neighbour_turbine_count = sum([x.Turbines.Count 
                                    for x in wf.Workbook.WindFarms 
                                    if x.IsNeighbour == False])
active_scenario_name = next(x.Name 
                        for x in wf.Workbook.LayoutScenarios 
                        if x.IsActive)
print(f'Scenario \"{active_scenario_name}\"\n has {turbine_count} turbines,\n of which {non_neighbour_turbine_count} are in your wind farms (not neighbours)')

Scenario "6D 250 degrees x 4.5D 20 degrees"
 has 41 turbines,
 of which 41 are in your wind farms (not neighbours)


### Compute Full AEP for every scenario in the workbook

In [5]:
result_string = "Results: \n"
success = True
for layout_scenario in wf.Workbook.LayoutScenarios:
    try:
        wf.Toolbox.ActivateLayoutScenario(layout_scenario)
        print (f'Activated scenario: \"{layout_scenario.Name}\"')
        
        # Set some energy calculation settings
        energySettings = wf.Workbook.ModelSettings.EnergySettings
        energySettings.WakeModelType =  wf.Scripting.WakeModelType.EddyViscosity
        energySettings.ApplyLargeWindFarmCorrection = True
        energySettings.CalculationToUse = wf.Scripting.EnergyCalculationToUseType.New
        energySettings.CalculateEfficiencies = False
        energySettings.LargeWindFarmCorrectionSettings.BaseRoughness = 0.0002
        energySettings.LargeWindFarmCorrectionSettings.IncreasedRoughness = 0.0192
        energySettings.LargeWindFarmCorrectionSettings.DistanceInDiametersToStartOfRecovery = 120
        energySettings.NumberOfDirectionSectors = 180
        
        # Compute the full yield
        print (f'> Computing energy...')
        results_scenario = wf.Toolbox.CalculateEnergy()
        print ('> ...Finished energy calculation!')
        
        # Read some results
        subjectWindFarm = [x for x in results_scenario.WindFarms if x.IsNeighbour == False][0]
        full_yeild = results_scenario.FarmTotalYields.GetVariantResult("Full").GetValueForFarm(subjectWindFarm).Value / 10e6	
        result = str.format("Full yield for scenario {0} =\t{1:.2f} GWh/annum\n", layout_scenario.Name, full_yeild)
        result_string += result
        print (result )

    except Exception as e:
        print(str(e))
        success = False

if (success):
    print("SUCCESS")
    print(result_string)
else:
    print("FAIL")

Activated scenario: "4D 160 degrees x 8D 250 degrees"
> Computing energy...
> ...Finished energy calculation!
Full yield for scenario 4D 160 degrees x 8D 250 degrees =	219.23 GWh/annum

Activated scenario: "6D 250 degrees x 4.5D 20 degrees"
> Computing energy...
> ...Finished energy calculation!
Full yield for scenario 6D 250 degrees x 4.5D 20 degrees =	309.26 GWh/annum

SUCCESS
Results: 
Full yield for scenario 4D 160 degrees x 8D 250 degrees =	219.23 GWh/annum
Full yield for scenario 6D 250 degrees x 4.5D 20 degrees =	309.26 GWh/annum



Close the open workbook:


In [6]:
wf.Toolbox.NewWorkbook()

# WindFarmer API demo: AnnualEnergyProduction
The WindFarmer API is a web API. You can call it from any coding language, or use tools like Postman. There is an OpenAPI definition which provides
documentation, and allows client code to be generated.

From python you can call the API directly, using `urllib3` or `requests`.

## Using the API directly
First, import the necessary modules

In [7]:
import os
import requests
import json
import time

To access the API you need a authorization token. This should be kept secure - and not added to source control, so I'm getting it from an environment variable.

The token should be passed as an Authorization header. We also need to set the `Content-Type` to let the API know that we're sending JSON data.

In [8]:
BASE_URL = 'https://windfarmer.dnv.com/api/v2/'
BEARER_TOKEN = os.environ['WINDFARMER_ACCESS_KEY']

headers = {
    'Authorization': f'Bearer {BEARER_TOKEN}',
    'Content-Type': 'application/json'
}

### Call `Status`
Try calling the `Status` endpoint. This verifies that you can access the API and that your token is valid.

In [9]:
response = requests.get(BASE_URL + 'Status', headers = headers)
print(f'Response from Status: {response.status_code}')
print(response.text)

Response from Status: 200
{"message":"Connection to DNV WindFarmer Services API was successful.","windFarmerServicesAPIVersion":"2.5.3","calculationLibraryVersion":"2.5.8.0"}


### Call `AnnualEnergyProduction`

#### Prepare the input data
This could come from elsewhere in your script, but here we're just loading it from a json file.
We'll first export that from the WindFarmer workbook from before

In [10]:
wf.Toolbox.OpenWorkbook(workbook_file_path)
active_scenario_name = next(x.Name 
                        for x in wf.Workbook.LayoutScenarios 
                        if x.IsActive)
aep_json_inputs_file_path = os.path.join(workbook_folder_path, 'BalticCoastLayout.json')
wf.Toolbox.ExportWindFarmerEnergyJson(aep_json_inputs_file_path)
# Close the workbook
wf.Toolbox.NewWorkbook()

Load the json inputs...

In [11]:
with open(aep_json_inputs_file_path) as f:
    json_string = f.read()
    input_data = json.loads(json_string)

print(json_string[0:500] +  '...')


{
  "projectInfo": null,
  "windFarms": [
    {
      "name": "Farm",
      "turbines": [
        {
          "windFarm": null,
          "name": "Turbine 1",
          "associatedWindClimateId": "FINO_2 150m",
          "turbineModelId": "IEA Wind 15-MW Turbine 240m_150mHH",
          "isInstalled": false,
          "productionYield": 0.0,
          "confidenceWeighting": 0.0,
          "location": {
            "easting_m": 4574986.191871113,
            "northing_m": 6057717.212971707,
      ...


The data is now in a dictionary...

In [12]:
input_data['windFarms'][0]['turbines'][0]['location']['easting_m']

4574986.191871113

Send the input data to the Annual Energy Production calculation

In [13]:
start = time.time()
response = requests.post(
    BASE_URL + 'AnnualEnergyProduction', 
    headers=headers,
    json = input_data)
    
print(f'Response {response.status_code} - {response.reason} in {time.time() - start:.2f}s')
# Print the error detail if we haven't receieved a 200 OK response
if response.status_code != 200:
    print(json.loads(response.content)['detail'])

Response 200 - OK in 1.22s


Deserialize the response, and use it...

In [14]:
result = json.loads(response.content)
full_aep_MWh_per_year = float(result['windFarmAepOutputs'][0]['fullAnnualEnergyYield_MWh_per_year'])
print(f'full Annual Energy Production = {full_aep_MWh_per_year/10e3:.2f} GWh / year')

full Annual Energy Production = 309.26 GWh / year


# Call the web API asynchronously
Example asynchronous calls to the API, 
modifying a single json file to describe the input by deleting one turbine at a time

In [15]:
# Using asyncio this time, not requests 
import aiohttp
import asyncio
import time
import os
import json

some settings...

In [16]:
number_of_tests = 10

# Input JSON files
api_inputs_file = aep_json_inputs_file_path 

# API connection settings (same as above)
BASE_URL = 'https://windfarmer.dnv.com/api/v2/'
BEARER_TOKEN = os.environ['WINDFARMER_ACCESS_KEY']

headers = {
    'Authorization': f'Bearer {BEARER_TOKEN}',
    'Content-Type': 'application/json'
}

In [17]:
# Define an asynchronous method to call the API 
async def call_api(session, api_name, id, data):
    print(f'{api_name} - call {id}')
    
    async with session.post(
        BASE_URL + api_name,
        headers = headers,
        json = data) as resp:
        
        result = await resp.json()
        print(f'{api_name} - call {id} complete with status {resp.status}')
        return result
        
# Generate and make the API calls
async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(0, number_of_tests):
            with open(api_inputs_file) as f:
                input_data = json.load(f)
                # delete one turbine to generate a new test case
                del input_data['windFarms'][0]['turbines'][i]
                tasks.append(asyncio.ensure_future(call_api(session, 'AnnualEnergyProduction', i, input_data)))

        all_results = await asyncio.gather(*tasks)
        for result in all_results:
            full_energy_yield = result['windFarmAepOutputs'][0]['fullAnnualEnergyYield_MWh_per_year'] / 10e3
            print(f"Full annual energy yield: {full_energy_yield:.2f} GWh/year")

start_time = time.time()
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
await main()
print(f'--- {time.time() - start_time:.2f} seconds ---')            

AnnualEnergyProduction - call 0
AnnualEnergyProduction - call 1
AnnualEnergyProduction - call 2
AnnualEnergyProduction - call 3
AnnualEnergyProduction - call 4
AnnualEnergyProduction - call 5
AnnualEnergyProduction - call 6
AnnualEnergyProduction - call 7
AnnualEnergyProduction - call 8
AnnualEnergyProduction - call 9
AnnualEnergyProduction - call 9 complete with status 200
AnnualEnergyProduction - call 1 complete with status 200
AnnualEnergyProduction - call 8 complete with status 200
AnnualEnergyProduction - call 0 complete with status 200
AnnualEnergyProduction - call 7 complete with status 200
AnnualEnergyProduction - call 2 complete with status 200
AnnualEnergyProduction - call 4 complete with status 200
AnnualEnergyProduction - call 6 complete with status 200
AnnualEnergyProduction - call 3 complete with status 200
AnnualEnergyProduction - call 5 complete with status 200
Full annual energy yield: 302.17 GWh/year
Full annual energy yield: 302.22 GWh/year
Full annual energy yield: 